In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# This is Luke's scratchpad notebook.

In [3]:
# SVI doesn't have zip codes, it has fips. So I need to transform FIPS into zip codes, and *then* I can join on the Covid cases onto the data frame

#### Key datasource:
https://www.huduser.gov/portal/datasets/usps_crosswalk.html
Possible note for presentation comments:
- we downloaded the 4th quarter 2018 data Tact-to-zip file, which is the file from HUD that'll translate the data

### Data Prep

In [4]:
bexar_covid = pd.read_csv('Bexar_County_COVID-19_Data_by_Zip_Code.csv')
bexar_covid.head(15)

,OBJECTID,ZIP_CODE,populationtotals_totpop_cy,Positive,CasesP100000,ZipInt,ActiveCases,ActiveCaseRate,SHAPE_Length,SHAPE_Area
0,1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
1,2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
2,4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
3,5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
4,7,78052,699,28,4005.722461,78052,1,143.061517,0.260085,0.001147
5,12,78069,2290,83,3624.454148,78069,5,218.340611,0.254859,0.002041
6,13,78073,7312,300,4102.844639,78073,14,191.466083,0.580798,0.011685
7,14,78101,4485,104,2318.840580,78101,7,156.075808,0.543172,0.006502
8,15,78108,880,7,795.454545,78108,1,113.636364,0.304887,0.000858
9,16,78109,50260,1346,2678.074015,78109,66,131.317151,0.737849,0.007682


In [5]:
bexar_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   OBJECTID                    74 non-null     int64  
 1   ZIP_CODE                    74 non-null     int64  
 2   populationtotals_totpop_cy  74 non-null     int64  
 3   Positive                    74 non-null     int64  
 4   CasesP100000                74 non-null     float64
 5   ZipInt                      74 non-null     int64  
 6   ActiveCases                 74 non-null     int64  
 7   ActiveCaseRate              74 non-null     float64
 8   SHAPE_Length                74 non-null     float64
 9   SHAPE_Area                  74 non-null     float64
dtypes: float64(4), int64(6)
memory usage: 5.9 KB


In [6]:
# Importing the SVI data into it's own DF:

bexar_svi = pd.read_csv('san_antonio_2018_tract.csv')
bexar_svi.columns = bexar_svi.columns.str.lower()
bexar_svi.head()

,st,state,st_abbr,stcnty,county,fips,location,area_sqmi,e_totpop,m_totpop,...,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,0,1,1,3,3,272,130,11.3,5.0,38328
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,0,1,0,2,4,606,206,20.0,5.7,9202
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,0,1,0,1,9,465,125,19.5,4.4,2044
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,0,1,1,3,8,690,265,24.5,7.4,6776
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,0,1,0,1,3,251,128,22.5,8.8,3716


In [7]:
bexar_svi.fips.nunique()

366

In [8]:
# Thanks to HUD's crosswalk dataset, we can convert tracts to zip codes:

tract_to_zip = pd.read_csv('tract_zip_122018_hud.csv')
tract_to_zip.head()

,fips,zip
0,1001020100,36067
1,1001020200,36067
2,1001020300,36067
3,1001020400,36067
4,1001020400,36066


In [16]:
tract_to_zip.fips.value_counts().value_counts()

1     27886
2     22869
3     10507
4      5164
5      2767
6      1688
7      1053
8       664
9       372
10      234
11      122
12       65
13       33
14       12
15       11
18        5
16        4
17        4
20        2
30        1
19        1
22        1
23        1
25        1
28        1
29        1
44        1
Name: fips, dtype: int64

In [17]:
tract_to_zip.fips.value_counts()

11001006202    44
11001010700    30
11001005800    29
11001010200    28
54011000600    25
               ..
9009125200      1
39049006923     1
34029722402     1
1101005609      1
29189218300     1
Name: fips, Length: 73470, dtype: int64

In [20]:
test_df = tract_to_zip[tract_to_zip.fips == 54011000600]

In [21]:
merge_test = pd.merge(bexar_svi, test_df, on = 'fips', how = 'left')

In [22]:
merge_test

,st,state,st_abbr,stcnty,county,fips,location,area_sqmi,e_totpop,m_totpop,...,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,NaN
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,1,0,2,4,606,206,20.0,5.7,9202,NaN
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,1,0,1,9,465,125,19.5,4.4,2044,NaN
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,1,1,3,8,690,265,24.5,7.4,6776,NaN
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,1,0,1,3,251,128,22.5,8.8,3716,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,48,TEXAS,TX,48029,Bexar,48029980002,"Census Tract 9800.02, Bexar County, Texas",0.601487,0,13,...,-999,-999,-999,-999,0,13,-999.0,-999.0,1264,NaN
362,48,TEXAS,TX,48029,Bexar,48029980003,"Census Tract 9800.03, Bexar County, Texas",6.233800,1133,265,...,0,1,1,1,11,11,1.8,1.8,1051,NaN
363,48,TEXAS,TX,48029,Bexar,48029980004,"Census Tract 9800.04, Bexar County, Texas",3.779799,0,13,...,-999,-999,-999,-999,0,13,-999.0,-999.0,5504,NaN
364,48,TEXAS,TX,48029,Bexar,48029980005,"Census Tract 9800.05, Bexar County, Texas",15.269322,8,14,...,0,0,0,-999,0,13,0.0,100.0,4,NaN


In [9]:
# Merging:

left_joined = pd.merge(bexar_svi, tract_to_zip, on = 'fips', how = 'inner')
left_joined.head()

,st,state,st_abbr,stcnty,county,fips,location,area_sqmi,e_totpop,m_totpop,...,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78207
1,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78212
2,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78296
3,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78291
4,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78294


In [12]:
left_joined.tail()

,st,state,st_abbr,stcnty,county,fips,location,area_sqmi,e_totpop,m_totpop,...,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip
735,48,TEXAS,TX,48029,Bexar,48029980100,"Census Tract 9801, Bexar County, Texas",5.667057,709,189,...,0,1,2,3,27,22,9.5,7.9,12898,78237
736,48,TEXAS,TX,48029,Bexar,48029980100,"Census Tract 9801, Bexar County, Texas",5.667057,709,189,...,0,1,2,3,27,22,9.5,7.9,12898,78243
737,48,TEXAS,TX,48029,Bexar,48029980100,"Census Tract 9801, Bexar County, Texas",5.667057,709,189,...,0,1,2,3,27,22,9.5,7.9,12898,78226
738,48,TEXAS,TX,48029,Bexar,48029980100,"Census Tract 9801, Bexar County, Texas",5.667057,709,189,...,0,1,2,3,27,22,9.5,7.9,12898,78236
739,48,TEXAS,TX,48029,Bexar,48029980100,"Census Tract 9801, Bexar County, Texas",5.667057,709,189,...,0,1,2,3,27,22,9.5,7.9,12898,78242


In [10]:
bexar_svi.shape, tract_to_zip.shape, left_joined.shape

((366, 124), (171501, 2), (740, 125))

In [11]:
tract_to_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171501 entries, 0 to 171500
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   fips    171501 non-null  int64
 1   zip     171501 non-null  int64
dtypes: int64(2)
memory usage: 2.6 MB
